# Converting the CMIP6 Data Set to the OpenVisus Visualization Format

This Jupyter notebook provides an example of how one might take a portion of the CMIP6 Data Set and convert it to the OpenVisus Visualization Format.

## Assumptions

1. You are running this notebook via [OSG's OSPool Notebooks service](https://notebook.ospool.osg-htc.org) as a user of ap40.uw.osg-htc.org.

2. You have run through the steps in [the setup notebook](cmip6_setup.ipynb).

3. You have selected the `openvisuspy` kernel for this notebook via the Jupyter interface.

## Ensure that job credentials are available

N.B. This step is necessary only to work around a bug or misconfiguration that has yet to be fixed. Normally, these credentials are handled by HTCondor automatically.

In [ ]:
!echo | condor_store_cred add-oauth -s scitokens -i -

## Submit the job

Set `ds_object` to the NetCDF file that you wish to convert.

Set `container_image` to the location of the `openvisuspy` container image that you created in [the setup notebook](cmip6_setup.ipynb).

In [ ]:
import htcondor
import pathlib


## Define the object to convert.

federation_prefix = "osdf://aws-opendata/us-west-2"
ds_object = "nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/hurs/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc"
destination_dir = "openvisus"


## Remove log files from previous runs.

for ext in [".log", ".out", ".err"]:
    pathlib.Path(f"convert_dataset{ext}").unlink(missing_ok=True)


## Submit the job.

job_description = htcondor.Submit(
    f"""
    container_image = osdf://ospool/ap40/data/brian.aydemir/openvisuspy-20240805-0945.sif
    args = python3 convert_dataset.py {destination_dir} $BASENAME({ds_object})

    transfer_input_files = convert_dataset.py, {federation_prefix}/{ds_object}
    transfer_output_files = {destination_dir}

    ## Save the job log, and standard output and error.
    log = convert_dataset.log
    output = convert_dataset.out
    error = convert_dataset.err

    ## Specify resource requests and other requirements.
    request_cpus = 2
    request_memory = 4G
    request_disk = 4G
    """
)

submitted_job = htcondor.Schedd().submit(job_description)

## Wait for the job to complete

In [ ]:
import demo_support

demo_support.wait_for_job(f"convert_dataset.log")